# ConversationBufferWindowMemory

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large

Let's first explore the basic functionality of this type of memory.

In [3]:
from langchain.memory import ConversationBufferWindowMemory

In [4]:
memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})

In [5]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [8]:
memory = ConversationBufferWindowMemory( k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.save_context({"input": "not much you"}, {"ouput": "not much"})

In [9]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you', additional_kwargs={}),
  AIMessage(content='not much', additional_kwargs={})]}

## Using in a chain
Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [7]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_summary.predict(input="你好，有什么事吗？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，有什么事吗？
AI:

> Finished chain.


' 你好！很高兴见到你！我可以为你做什么？我可以提供信息，提供建议，或者帮助你解决问题。'

In [8]:
conversation_with_summary.predict(input="他们有什么问题？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，有什么事吗？
AI:  你好！很高兴见到你！我可以为你做什么？我可以提供信息，提供建议，或者帮助你解决问题。
Human: 他们有什么问题？
AI:

> Finished chain.


' 对不起，我不知道。你可以更具体地描述一下你的问题，我可以尝试帮助你。'

In [9]:
conversation_with_summary.predict(input="进展顺利吗？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，有什么事吗？
AI:  你好！很高兴见到你！我可以为你做什么？我可以提供信息，提供建议，或者帮助你解决问题。
Human: 他们有什么问题？
AI:  对不起，我不知道。你可以更具体地描述一下你的问题，我可以尝试帮助你。
Human: 进展顺利吗？
AI:

> Finished chain.


' 对不起，我不知道。你可以更具体地描述一下你的问题，我可以尝试帮助你。'

In [10]:
# Notice here that the first interaction does not appear.
conversation_with_summary.predict(input="解决方案是什么？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 他们有什么问题？
AI:  对不起，我不知道。你可以更具体地描述一下你的问题，我可以尝试帮助你。
Human: 进展顺利吗？
AI:  对不起，我不知道。你可以更具体地描述一下你的问题，我可以尝试帮助你。
Human: 解决方案是什么？
AI:

> Finished chain.


' 对不起，我不知道。你可以更具体地描述一下你的问题，我可以尝试帮助你。'

![202305042306798.png](https://s2.loli.net/2023/05/04/geowAHJspzYaWx1.png)